### Deps

In [ ]:
import re
import requests
import random
import time
import pandas as pd
from bs4 import BeautifulSoup

### Extração dos links

In [ ]:
def sleep_random(min_ms=10, max_ms=2000):
    tempo_ms = random.randint(min_ms, max_ms)
    time.sleep(tempo_ms / 1000)

In [ ]:
def get_soup_nav(page) -> BeautifulSoup:
        url = "https://mcreator.net/modifications?page={}#google_vignette".format(page)
        response = requests.get(url)

        if response.status_code == 200:
            return BeautifulSoup(response.text, 'html.parser')
        
        print("Failed to retrieve page {}: {}".format(page, response.status_code))
        return None

In [ ]:
def get_links(soup: BeautifulSoup) -> list:
    links = []
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            links.append(href)
    return links

In [ ]:
def get_all_links() -> list:
    page = 1
    all_links = []

    while True:
        sleep_random()

        soup = get_soup_nav(page)
        if soup is None:
            break

        links = get_links(soup)

        pattern = re.compile(r'.*modification/\d+/.*', re.IGNORECASE)
        filtered_links = [link for link in links if pattern.match(link)]

        if not filtered_links:
            print("No more links found on page {}".format(page))
            continue
        else:
            print("Found {} links on page {}".format(len(filtered_links), page))
        
        all_links.extend(filtered_links)

        if page == 2:
            print("Stopping after 2 pages.")
            break

        page += 1
    return all_links

### Extração dos meta-dados

In [ ]:
def get_soup_mod(link) -> BeautifulSoup:
    url = "https://mcreator.net" + link
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.text, 'html.parser')
    
    print("Failed to retrieve mod page: {}, link {}".format(response.status_code, link))
    return None

In [ ]:
def get_headers(soup: BeautifulSoup) -> dict:
    h1_tag = soup.find('h1')
    return h1_tag.text.strip() if h1_tag else None

In [ ]:
def get_data_mod(soup: BeautifulSoup) -> dict:
    data = {'name': '',
            'description': '',
            'author': '',
            'date_published': '',
            'date_updated': '',
            'downloads': 0,
            'views': 0,
            'version': '',
            'category': '',
            'tags': [],
            'modloader': '',
            'amount_updates': 0,
            'size': 0,
            'dependencies': [],
            'link': [],
            'source':''}
    
    data['name'] = get_headers(soup)
    print("Name: {}".format(data['name']))
    return data

In [ ]:
def get_data_mcreator_net() -> pd.DataFrame:
    links = get_all_links()
    mods = []
    for link in links:
        sleep_random(0, 500)
        soup = get_soup_mod(link)
        if soup is None:
            continue

        data = get_data_mod(soup)
        if data is None:
            continue
        mods.append(data)
    
    df = pd.DataFrame(mods)
    return df

In [ ]:
df = get_data_mcreator_net()